In [1]:
import sys
sys.path.append('../../../../')

In [2]:
%load_ext autoreload
%autoreload 2

import torch
from jtvae import JTVAE
from datautils import MolTreeFolder, PairTreeFolder, MolTreeDataset

path_adj = "./"

/data/bora.oztekin/anaconda3/envs/jtvae37f/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry const> already registered; second conversion method ignored.
  return f(*args, **kwds)


In [3]:
from rdkit import Chem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

In [4]:
from dig.ggraph.dataset import ZINC800
from dig.ggraph.evaluation import Rand_Gen_Evaluator

In [5]:
dataset = ZINC800(one_shot=False, root=path_adj+'./datasets')

In [6]:
zinc_800_jt = torch.load(path_adj+"datasets/zinc_800_jt/processed/data.pt")

In [7]:
smiles = zinc_800_jt[-1]

In [8]:
# Define model
jtvae = JTVAE(smiles)

/data/bora.oztekin/anaconda3/envs/jtvae37f/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [9]:
preprocessed = jtvae.preprocess(smiles[:20])

100%|██████████| 20/20 [00:05<00:00,  3.63it/s]


In [10]:
loader = MolTreeFolder(preprocessed, jtvae.vocab, 32, num_workers=4)
load_epoch = 0
lr = 1e-3
anneal_rate = 0.9
clip_norm = 50.0
num_epochs = 1
total_step = 0  # TODO args.load_epoch
beta = 0.0  # TODO args.beta
jtvae.train_rand_gen(loader, load_epoch, lr, anneal_rate, clip_norm, num_epochs)

Model #Params: 4862K


/data/bora.oztekin/anaconda3/envs/jtvae37f/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


In [11]:
samples = list(map(Chem.MolFromSmiles, jtvae.run_rand_gen(30)))

/data/bora.oztekin/anaconda3/envs/jtvae37f/lib/python3.7/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/data/bora.oztekin/anaconda3/envs/jtvae37f/lib/python3.7/site-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [12]:
res_dict = {'mols': samples, 'train_smiles': smiles}
evaluator = Rand_Gen_Evaluator()
results = evaluator.eval(res_dict)
print(results)

Valid Ratio: 30/30 = 100.00%
Unique Ratio: 21/30 = 70.00%
Novel Ratio: 30/30 = 100.00%
{'valid_ratio': 100.0, 'unique_ratio': 70.0, 'novel_ratio': 100.0}
